In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy
from pyproj import Proj

In [2]:
def rounded_to_the_last_30_minute():
    now = datetime.now()
    rounded = now - (now - datetime.min) % timedelta(minutes=30)
    return rounded

In [3]:
date = rounded_to_the_last_30_minute()
date

datetime.datetime(2023, 11, 6, 3, 0)

In [4]:
YYYYMMDD_HHMM = date.strftime('%Y%m%d_%H%M')
YYYYMMDD_HHMM

'20231106_0300'

In [5]:
File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"
File

'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_20231106_0300.grib2'

In [6]:
ds = xr.open_dataset(File)
#ds

In [7]:
ds = ds.metpy.parse_cf()
ds = ds.metpy.assign_latitude_longitude(force=False)
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 3,
                                                                               time_bounds_1: 2,
                                                                               time1: 6,
                                                                               time1_bounds_1: 2,
                                                                               time3: 3,
                                                                               ...
                                                                               time4_bounds_1: 2,
                                                                               x: 2145,
                                                                               y: 1377,
                                                                               time2: 47,
                                                                               height_above_ground1: 1,
                                                                               height_above_ground: 1)
Coordinates: (12/13)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
  * time3                                                                     (time3) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * x                                                                         (x) float32 ...
    ...                                                                        ...
  * time2                                                                     (time2) datetime64[ns] ...
  * height_above_ground1                                                      (height_above_ground1) float32 ...
    metpy_crs                                                                 object ...
  * height_above_ground                                                       (height_above_ground) float32 ...
    latitude                                                                  (y, x) float64 ...
    longitude                                                                 (y, x) float64 ...
Dimensions without coordinates: time_bounds_1, time1_bounds_1, time3_bounds_1,
                                time4_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time1_bounds                                                              (time1, time1_bounds_1) datetime64[ns] ...
    time3_bounds                                                              (time3, time3_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time2, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time4, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time2, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time2, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time2, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                       

In [8]:
x, y = ds.x, ds.y

In [9]:
max_temp = ds.Maximum_temperature_height_above_ground_12_Hour_Maximum

In [10]:
proj_data = max_temp.metpy.cartopy_crs
proj_data;

In [11]:
pFull = Proj(proj_data)

In [12]:
def find_closest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [13]:
siteName = "ETEC"
siteLat, siteLon = (42.68, -73.81) #lat & lon of gridpoint over ETEC
siteX, siteY = pFull(siteLon, siteLat)
siteXidx, siteYidx = find_closest(x, siteX), find_closest(y, siteY)

In [14]:
precip6hr = ds.Total_precipitation_surface_6_Hour_Accumulation
precip6hr = precip6hr.isel(x = siteXidx, y = siteYidx).isel()
precip6hr

<xarray.DataArray 'Total_precipitation_surface_6_Hour_Accumulation' (time4: 12)>
[12 values with dtype=float32]
Coordinates:
    reftime    datetime64[ns] 2023-11-06T03:00:00
  * time4      (time4) datetime64[ns] 2023-11-06T06:00:00 ... 2023-11-09
    x          float32 1.816e+06
    y          float32 2.141e+06
    metpy_crs  object Projection: lambert_conformal_conic
    latitude   float64 42.68
    longitude  float64 -73.81
Attributes: (12/14)
    long_name:                       Total precipitation (6_Hour Accumulation...
    units:                           kg.m-2
    description:                     Total precipitation
    grid_mapping:                    LambertConformal_Projection
    Grib_Statistical_Interval_Type:  Accumulation
    Grib_Variable_Id:                VAR_0-1-8_L1_I6_Hour_S1
    ...                              ...
    Grib2_Parameter_Category:        Moisture
    Grib2_Parameter_Name:            Total precipitation
    Grib2_Level_Type:                1
    Grib2_Level_Desc:                Ground or water surface
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  Accumulation

In [15]:
times6 = precip6hr.metpy.time.values
times6

array(['2023-11-06T06:00:00.000000000', '2023-11-06T12:00:00.000000000',
       '2023-11-06T18:00:00.000000000', '2023-11-07T00:00:00.000000000',
       '2023-11-07T06:00:00.000000000', '2023-11-07T12:00:00.000000000',
       '2023-11-07T18:00:00.000000000', '2023-11-08T00:00:00.000000000',
       '2023-11-08T06:00:00.000000000', '2023-11-08T12:00:00.000000000',
       '2023-11-08T18:00:00.000000000', '2023-11-09T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [16]:
wx = ds.Weather_string_surface
wx = wx.isel(x = siteXidx, y = siteYidx).isel()
wx

<xarray.DataArray 'Weather_string_surface' (time2: 47)>
[47 values with dtype=float32]
Coordinates:
    reftime    datetime64[ns] 2023-11-06T03:00:00
    x          float32 1.816e+06
    y          float32 2.141e+06
  * time2      (time2) datetime64[ns] 2023-11-06T04:00:00 ... 2023-11-09
    metpy_crs  object Projection: lambert_conformal_conic
    latitude   float64 42.68
    longitude  float64 -73.81
Attributes: (12/13)
    long_name:                       Weather string @ Ground or water surface
    units:                           
    abbreviation:                    Wx
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_0-1-192_L1
    Grib2_Parameter:                 [  0   1 192]
    ...                              ...
    Grib2_Parameter_Category:        Moisture
    Grib2_Parameter_Name:            Weather string
    Grib2_Level_Type:                1
    Grib2_Level_Desc:                Ground or water surface
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [17]:
wx.values

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3., 19.,  4.,  4.,
       25.,  4.,  4., 23., 22.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.], dtype=float32)

In [18]:
forecastMax = max_temp.isel(x = siteXidx, y = siteYidx).isel()
forecastMax

<xarray.DataArray 'Maximum_temperature_height_above_ground_12_Hour_Maximum' (
                                                                             time3: 3,
                                                                             height_above_ground1: 1)>
[3 values with dtype=float32]
Coordinates:
    reftime               datetime64[ns] 2023-11-06T03:00:00
  * time3                 (time3) datetime64[ns] 2023-11-07 ... 2023-11-09
    x                     float32 1.816e+06
    y                     float32 2.141e+06
  * height_above_ground1  (height_above_ground1) float32 2.0
    metpy_crs             object Projection: lambert_conformal_conic
    latitude              float64 42.68
    longitude             float64 -73.81
Attributes: (12/14)
    long_name:                       Maximum temperature (12_Hour Maximum) @ ...
    units:                           K
    description:                     Maximum temperature
    grid_mapping:                    LambertConformal_Projection
    Grib_Statistical_Interval_Type:  Maximum
    Grib_Variable_Id:                VAR_0-0-4_L103_I12_Hour_S2
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Maximum temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  Maximum

In [19]:
timeDimMax, vertDimMax = forecastMax.metpy.time.name, forecastMax.metpy.vertical.name
timeDimMax, vertDimMax

('time3', 'height_above_ground1')

In [20]:
idxTimeTemp = slice(None, 2) # First time
idxVertTemp = 0 # First (and in this case, only) vertical level

timeDictMax = {timeDimMax: idxTimeTemp}
vertDictMax = {vertDimMax: idxVertTemp}

timeDictMax, vertDictMax

({'time3': slice(None, 2, None)}, {'height_above_ground1': 0})

In [21]:
forecastMax = forecastMax.isel(vertDictMax).isel(timeDictMax)
forecastMax = forecastMax.metpy.convert_units('degF')
forecastMax

Magnitude,[50.99001693725586 63.049983978271484]
Units,degree_Fahrenheit


In [22]:
min_temp = ds.Minimum_temperature_height_above_ground_12_Hour_Minimum

In [23]:
forecastMin = min_temp.isel(x = siteXidx, y = siteYidx).isel()

In [24]:
timeDimMin, vertDimMin = forecastMin.metpy.time.name, forecastMin.metpy.vertical.name
timeDimMin, vertDimMin

('time', 'height_above_ground1')

In [25]:
timeDictMin = {timeDimMin: idxTimeTemp}
vertDictMin = {vertDimMin: idxVertTemp}

timeDictMin, vertDictMin

({'time': slice(None, 2, None)}, {'height_above_ground1': 0})

In [26]:
forecastMin = forecastMin.isel(vertDictMin).isel(timeDictMin)
forecastMin = forecastMin.metpy.convert_units('degF')
forecastMin

Magnitude,[34.069976806640625 43.969974517822266]
Units,degree_Fahrenheit


In [27]:
precip = ds.Total_precipitation_surface_12_Hour_Accumulation_probability_above_0p254

In [28]:
forecastPrecip = precip.isel(x = siteXidx, y = siteYidx).isel()

In [29]:
timeDimPrecip = forecastPrecip.metpy.time.name

In [30]:
idxTimeFull = slice(None, 4) # First 4 times

timeDictPrecip = {timeDimPrecip: idxTimeFull}

timeDictPrecip

{'time1': slice(None, 4, None)}

In [31]:
forecastPrecip = forecastPrecip.isel(timeDictPrecip)
forecastPrecip

<xarray.DataArray 'Total_precipitation_surface_12_Hour_Accumulation_probability_above_0p254' (
                                                                                              time1: 4)>
[4 values with dtype=float32]
Coordinates:
    reftime    datetime64[ns] 2023-11-06T03:00:00
  * time1      (time1) datetime64[ns] 2023-11-06T12:00:00 ... 2023-11-08
    x          float32 1.816e+06
    y          float32 2.141e+06
    metpy_crs  object Projection: lambert_conformal_conic
    latitude   float64 42.68
    longitude  float64 -73.81
Attributes: (12/16)
    long_name:                       Probability Total precipitation (12_Hour...
    units:                           %
    description:                     Total precipitation
    grid_mapping:                    LambertConformal_Projection
    Grib_Statistical_Interval_Type:  Accumulation
    Grib_Variable_Id:                VAR_0-1-8_L1_I12_Hour_S1_Prob_above_0p254
    ...                              ...
    Grib2_Level_Type:                1
    Grib2_Level_Desc:                Ground or water surface
    Grib2_Probability_Type:          1
    Grib2_Probability_Name:          above_0.254
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  Accumulation

In [32]:
times12 = forecastPrecip.metpy.time.values
times12

array(['2023-11-06T12:00:00.000000000', '2023-11-07T00:00:00.000000000',
       '2023-11-07T12:00:00.000000000', '2023-11-08T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [33]:
cloudCover = ds.Total_cloud_cover_surface

In [34]:
forecastCloudCover = cloudCover.isel(x = siteXidx, y = siteYidx).isel()

In [35]:
timeDimCloudCover = forecastCloudCover.metpy.time.name

In [36]:
idxTimeCloudCover = slice(None, 48)

timeDictCloudCover = {timeDimCloudCover: idxTimeCloudCover}
timeDictCloudCover

{'time2': slice(None, 48, None)}

In [37]:
forecastCloudCover = forecastCloudCover.isel(timeDictCloudCover)
forecastCloudCover

<xarray.DataArray 'Total_cloud_cover_surface' (time2: 47)>
[47 values with dtype=float32]
Coordinates:
    reftime    datetime64[ns] 2023-11-06T03:00:00
    x          float32 1.816e+06
    y          float32 2.141e+06
  * time2      (time2) datetime64[ns] 2023-11-06T04:00:00 ... 2023-11-09
    metpy_crs  object Projection: lambert_conformal_conic
    latitude   float64 42.68
    longitude  float64 -73.81
Attributes: (12/13)
    long_name:                       Total cloud cover @ Ground or water surface
    units:                           %
    description:                     Total cloud cover
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_0-6-1_L1
    Grib2_Parameter:                 [0 6 1]
    ...                              ...
    Grib2_Parameter_Category:        Cloud
    Grib2_Parameter_Name:            Total cloud cover
    Grib2_Level_Type:                1
    Grib2_Level_Desc:                Ground or water surface
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [38]:
forecastCloudCover.values

array([21., 23., 25., 33., 40., 47., 51., 54., 58., 59., 61., 62., 67.,
       72., 77., 76., 74., 73., 73., 73., 73., 79., 83., 92., 85., 82.,
       79., 77., 76., 74., 68., 63., 57., 57., 57., 56., 69., 57., 67.,
       70., 68., 52., 26., 21., 26., 35., 58.], dtype=float32)

In [39]:
times1 = forecastCloudCover.metpy.time.values
times1

array(['2023-11-06T04:00:00.000000000', '2023-11-06T05:00:00.000000000',
       '2023-11-06T06:00:00.000000000', '2023-11-06T07:00:00.000000000',
       '2023-11-06T08:00:00.000000000', '2023-11-06T09:00:00.000000000',
       '2023-11-06T10:00:00.000000000', '2023-11-06T11:00:00.000000000',
       '2023-11-06T12:00:00.000000000', '2023-11-06T13:00:00.000000000',
       '2023-11-06T14:00:00.000000000', '2023-11-06T15:00:00.000000000',
       '2023-11-06T16:00:00.000000000', '2023-11-06T17:00:00.000000000',
       '2023-11-06T18:00:00.000000000', '2023-11-06T19:00:00.000000000',
       '2023-11-06T20:00:00.000000000', '2023-11-06T21:00:00.000000000',
       '2023-11-06T22:00:00.000000000', '2023-11-06T23:00:00.000000000',
       '2023-11-07T00:00:00.000000000', '2023-11-07T01:00:00.000000000',
       '2023-11-07T02:00:00.000000000', '2023-11-07T03:00:00.000000000',
       '2023-11-07T04:00:00.000000000', '2023-11-07T05:00:00.000000000',
       '2023-11-07T06:00:00.000000000', '2023-11-07

In [40]:
def skyConditionFromCloudCover(cloudCover):
    if 0 <= cloudCover <= 5:
        skyCondition = 'Clear'
    elif 6 <= cloudCover <= 25:
        skyCondition = 'Mostly Clear'
    elif 26 <= cloudCover <= 50:
        skyCondition = 'Partly Cloudy'
    elif 51 <= cloudCover <= 69:
        skyCondition = 'Mostly Cloudy'
    elif 70 <= cloudCover <= 87:
        skyCondition = 'Considerable Cloudiness'
    elif 88 <= cloudCover <= 100:
        skyCondition = 'Overcast'
    return skyCondition

In [41]:
def precipProbFromPOP(POP):
    if 0 <= POP <= 19:
        precipProb = 'None'
    elif 20 <= POP <= 29:
        precipProb = 'Slight Chance'
    elif 30 <= POP <= 59:
        precipProb = 'Chance'
    elif 60 <= POP <= 79:
        precipProb = 'Likely'
    elif 80 <= POP <= 100:
        precipProb = 'Certain'
    return precipProb

In [42]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [43]:
iCloud = 0

cloudCoverList_11 = []

for cloud_time_step in times1:
    
    
    if cloud_time_step <= times6[0]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_11.append(skyCover)
     
    iCloud = iCloud + 1

In [44]:
cloudCoverList_11

['Mostly Clear', 'Mostly Clear', 'Mostly Clear']

In [45]:
cc_11 = most_common(cloudCoverList_11)
cc_11

'Mostly Clear'

In [46]:
iCloud = 0

cloudCoverList_12 = []

for cloud_time_step in times1:
    
    
    if times6[0] < cloud_time_step <= times6[1]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_12.append(skyCover)
     
    iCloud = iCloud + 1

In [47]:
cloudCoverList_12

['Partly Cloudy',
 'Partly Cloudy',
 'Partly Cloudy',
 'Mostly Cloudy',
 'Mostly Cloudy',
 'Mostly Cloudy']

In [48]:
cc_12 = most_common(cloudCoverList_12)
cc_12

'Mostly Cloudy'

In [49]:
iCloud = 0

cloudCoverList_21 = []

for cloud_time_step in times1:
    
    
    if times6[1] < cloud_time_step <= times6[2]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_21.append(skyCover)
     
    iCloud = iCloud + 1

In [50]:
cloudCoverList_21

['Mostly Cloudy',
 'Mostly Cloudy',
 'Mostly Cloudy',
 'Mostly Cloudy',
 'Considerable Cloudiness',
 'Considerable Cloudiness']

In [51]:
cc_21 = most_common(cloudCoverList_21)
cc_21

'Mostly Cloudy'

In [52]:
iCloud = 0

cloudCoverList_22 = []

for cloud_time_step in times1:
    
    
    if times6[2] < cloud_time_step <= times6[3]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_22.append(skyCover)
     
    iCloud = iCloud + 1

In [53]:
cloudCoverList_22

['Considerable Cloudiness',
 'Considerable Cloudiness',
 'Considerable Cloudiness',
 'Considerable Cloudiness',
 'Considerable Cloudiness',
 'Considerable Cloudiness']

In [54]:
cc_22 = most_common(cloudCoverList_22)
cc_22

'Considerable Cloudiness'

In [55]:
iCloud = 0

cloudCoverList_31 = []

for cloud_time_step in times1:
    
    
    if times6[3] < cloud_time_step <= times6[4]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_31.append(skyCover)
     
    iCloud = iCloud + 1

In [56]:
cloudCoverList_31

['Considerable Cloudiness',
 'Considerable Cloudiness',
 'Overcast',
 'Considerable Cloudiness',
 'Considerable Cloudiness',
 'Considerable Cloudiness']

In [57]:
cc_31 = most_common(cloudCoverList_31)
cc_31

'Considerable Cloudiness'

In [58]:
iCloud = 0

cloudCoverList_32 = []

for cloud_time_step in times1:
    
    
    if times6[4] < cloud_time_step <= times6[5]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_32.append(skyCover)
     
    iCloud = iCloud + 1

In [59]:
cloudCoverList_32

['Considerable Cloudiness',
 'Considerable Cloudiness',
 'Considerable Cloudiness',
 'Mostly Cloudy',
 'Mostly Cloudy',
 'Mostly Cloudy']

In [60]:
cc_32 = most_common(cloudCoverList_32)
cc_32

'Considerable Cloudiness'

In [61]:
iCloud = 0

cloudCoverList_41 = []

for cloud_time_step in times1:
    
    
    if times6[5] < cloud_time_step <= times6[6]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_41.append(skyCover)
     
    iCloud = iCloud + 1

In [62]:
cloudCoverList_41

['Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudy']

In [63]:
cc_41 = most_common(cloudCoverList_41)
cc_41

'Mostly Cloudy'

In [64]:
iCloud = 0

cloudCoverList_42 = []

for cloud_time_step in times1:
    
    
    if times6[6] < cloud_time_step <= times6[7]:
        timeDictCloudCover = {timeDimCloudCover: iCloud}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
        skyCover = skyConditionFromCloudCover(forecastCloudCoverNew)
    
        cloudCoverList_42.append(skyCover)
     
    iCloud = iCloud + 1

In [65]:
cloudCoverList_42

['Mostly Cloudy', 'Mostly Cloudy']

In [66]:
cc_42 = most_common(cloudCoverList_42)
cc_42

'Mostly Cloudy'

In [68]:
i=0
imax=0
imin=0

print('----------------')

for time_step in times12:
    
    period = i+1
    periodStr = str(period)
    
    print('Forecast Period: '+periodStr)
    
    timeStr = pd.to_datetime(str(time_step)) 
    timeStr = timeStr.strftime('%Y-%m-%d %H%M UTC')
    
    print('Valid through: '+timeStr)
    
    timeDictMax = {timeDimMax: imax}
    timeDictMin = {timeDimMin: imin}
    forecastMaxNew = forecastMax.isel(timeDictMax)
    forecastMinNew = forecastMin.isel(timeDictMin)
    
    if forecastMaxNew.metpy.time.values == time_step:
        roundedMax = round(forecastMaxNew.item(0), 1)
        maxStr = str(roundedMax)
        print('Forecast High: '+maxStr)
        if imax < 1:
            imax = imax+1
    
    elif forecastMinNew.metpy.time.values  == time_step:
        roundedMin = round(forecastMinNew.item(0), 1)
        minStr = str(roundedMin)
        print('Forecast Low: '+minStr)
        if imin < 1:
            imin = imin+1
    
    if i == 0:
        if cc_11 == cc_12:
            print (cc_11)
        else:
            print (cc_11 + ' --> ' + cc_12)
            
    elif i == 1:
        if cc_21 == cc_22:
            print (cc_21)
        else:
            print (cc_21 + ' --> ' + cc_22)
    
    elif i == 2:
        if cc_31 == cc_32:
            print (cc_31)
        else:
            print (cc_31 + ' --> ' + cc_32)
            
    elif i == 3:
        if cc_41 == cc_42:
            print (cc_41)
        else:
            print (cc_41 + ' --> ' + cc_42)   
    
    timeDictPrecip = {timeDimPrecip: i}
    forecastPrecipNew = forecastPrecip.isel(timeDictPrecip)
    precipStr = str(forecastPrecipNew.item(0))
    print('Probability of Measurable Precip: '+precipStr+'%')
    
    print('----------------')
    
    i=i+1  

----------------
Forecast Period: 1
Valid through: 2023-11-06 1200 UTC
Forecast Low: 34.1 degree_Fahrenheit
Mostly Clear --> Mostly Cloudy
Probability of Measurable Precip: 1.0%
----------------
Forecast Period: 2
Valid through: 2023-11-07 0000 UTC
Forecast High: 51.0 degree_Fahrenheit
Mostly Cloudy --> Considerable Cloudiness
Probability of Measurable Precip: 10.0%
----------------
Forecast Period: 3
Valid through: 2023-11-07 1200 UTC
Forecast Low: 44.0 degree_Fahrenheit
Considerable Cloudiness
Probability of Measurable Precip: 36.0%
----------------
Forecast Period: 4
Valid through: 2023-11-08 0000 UTC
Forecast High: 63.0 degree_Fahrenheit
Mostly Cloudy
Probability of Measurable Precip: 13.0%
----------------
